In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

import wrangle as w

In [9]:
# acquire data
w.acquire_df()
df = w.prepare_df()

In [10]:
df[df.user_id == 0]

,cohort_id,cohort_name,start_date,end_date,program_id,dates,time,endpoint,user_id,ip


In [22]:
# preview data
df[df.cohort_id == 1].user_id.unique()

array([ 11,  60,  64,  95, 141, 146, 148, 170, 209, 211, 212, 248, 257,
       312, 313, 316, 398, 402])

In [ ]:
# check for staff

In [12]:
# df info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900222 entries, 0 to 900221
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   cohort_id    900222 non-null  float64       
 1   cohort_name  847329 non-null  object        
 2   start_date   847329 non-null  datetime64[ns]
 3   end_date     847329 non-null  datetime64[ns]
 4   program_id   847329 non-null  float64       
 5   dates        900222 non-null  datetime64[ns]
 6   time         900222 non-null  object        
 7   endpoint     900222 non-null  object        
 8   user_id      900222 non-null  int64         
 9   ip           900222 non-null  object        
dtypes: datetime64[ns](3), float64(2), int64(1), object(4)
memory usage: 75.5+ MB


In [13]:
# df describe 
df.describe()

,cohort_id,program_id,user_id
count,900222.000000,847329.000000,900222.000000
mean,45.651338,2.086003,458.825565
std,33.800052,0.388230,249.296870
min,0.000000,1.000000,1.000000
25%,26.000000,2.000000,269.000000
50%,33.000000,2.000000,475.000000
75%,57.000000,2.000000,660.000000
max,139.000000,4.000000,981.000000


In [14]:
# view nulls
df.isnull().sum()

# 1 null in endpoint and 52893 in cohort

cohort_id          0
cohort_name    52893
start_date     52893
end_date       52893
program_id     52893
dates              0
time               0
endpoint           0
user_id            0
ip                 0
dtype: int64

In [15]:
# view endpoint null 2020-04-08 09:25:18 user: 586 cohort: 55.0 ip: 72.177.240.51
df[df.endpoint.isnull()]

,cohort_id,cohort_name,start_date,end_date,program_id,dates,time,endpoint,user_id,ip


In [26]:
user_cohort_null_list

array([], dtype=int64)

In [23]:
# view cohort nulls 78 unique users with cohort nulls -- possibly instructors or admin
cohort_nulls = df[df.cohort_id.isnull()]
user_cohort_null_list = cohort_nulls.user_id.unique()

In [27]:
# checks to see if user id has nothing but nans for cohort and places into nan_only list or nan_and_other
nan_only = []
nan_and_other = []

for u in user_cohort_null_list:
    null_cohort = df[df.user_id == u].cohort.unique()
    
    if np.isnan(null_cohort).all():
        nan_only.append(u)
    else:
        nan_and_other.append(u)


In [28]:
# there are 8 
len(nan_and_other)
nan_and_other

[]

In [ ]:
# check ids and cohorts
for u in nan_and_other:
    print(u)
    print(df[df.user_id == u].cohort.unique())
    
# 358, 375, 644, 744 only had one cohort
# all other ids have multiple cohorts

In [ ]:
# there are 70 ids with no cohort attached
len(nan_only)

**Question**
> If someone graduates will they always have that same cohort attached to them even after graduation?

> how many cohorts are there?

> how many people are in each cohort?


In [42]:
# how many cohorts are there?
# 47 cohorts minus the nan

# list of cohorts
cohort_list = list(df.cohort_id.unique())

In [33]:
# cohort names ids
cohort_df = pd.read_csv("curriculum_logs_cohorts.csv")

In [36]:
# how many people are in each cohort?
# number cohort and number of users
for n in cohort_list:
    print(f"Cohort: {n}")
    users_of_cohort = df[df.cohort_name == n].user_id.unique()
    print(f"Number of Users: {len(users_of_cohort)}")


Cohort: Hampton
Number of Users: 5
Cohort: Teddy
Number of Users: 35
Cohort: Sequoia
Number of Users: 28
Cohort: Arches
Number of Users: 18
Cohort: Niagara
Number of Users: 8
Cohort: Pinnacles
Number of Users: 14
Cohort: Quincy
Number of Users: 17
Cohort: nan
Number of Users: 0
Cohort: Kings
Number of Users: 10
Cohort: Lassen
Number of Users: 13
Cohort: Mammoth
Number of Users: 5
Cohort: Glacier
Number of Users: 6
Cohort: Denali
Number of Users: 1
Cohort: Joshua
Number of Users: 5
Cohort: Olympic
Number of Users: 14
Cohort: Ulysses
Number of Users: 33
Cohort: Badlands
Number of Users: 3
Cohort: Apollo
Number of Users: 1
Cohort: Ike
Number of Users: 5
Cohort: Voyageurs
Number of Users: 34
Cohort: Wrangell
Number of Users: 26
Cohort: Xanadu
Number of Users: 20
Cohort: Franklin
Number of Users: 2
Cohort: Yosemite
Number of Users: 29
Cohort: Staff
Number of Users: 51
Cohort: Zion
Number of Users: 25
Cohort: Andromeda
Number of Users: 28
Cohort: Betelgeuse
Number of Users: 22
Cohort: Ceres


In [49]:
df.user_id.nunique()

981

In [48]:
df.user_id.max()

981

In [ ]:
df.start_date

In [43]:
# user_ids in each cohort
cohort_dict = {}

for n in cohort_list:
    users_of_cohort = df[df.cohort_id 1== n].user_id.unique()
    users_of_cohort = list(users_of_cohort) 
    cohort_dict[n] = users_of_cohort

cohort_dict

{8.0: [1, 114, 150, 174, 646],
 22.0: [2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  12,
  13,
  14,
  18,
  19,
  20,
  23,
  25,
  29,
  30,
  31,
  32,
  34,
  35,
  39,
  43,
  44,
  46,
  99,
  138,
  142,
  154,
  155,
  180,
  254,
  284,
  459],
 21.0: [10,
  15,
  16,
  17,
  22,
  24,
  27,
  28,
  33,
  36,
  38,
  41,
  42,
  45,
  47,
  49,
  52,
  55,
  56,
  57,
  65,
  110,
  115,
  153,
  175,
  255,
  286,
  346],
 1.0: [11,
  60,
  64,
  95,
  141,
  146,
  148,
  170,
  209,
  211,
  212,
  248,
  257,
  312,
  313,
  316,
  398,
  402],
 16.0: [21, 40, 97, 143, 165, 181, 216, 322],
 18.0: [26, 67, 76, 81, 83, 84, 91, 104, 108, 162, 178, 179, 217, 611],
 19.0: [37,
  70,
  71,
  75,
  87,
  92,
  94,
  96,
  105,
  109,
  112,
  163,
  311,
  399,
  428,
  486,
  610],
 0.0: [48,
  54,
  58,
  59,
  61,
  62,
  63,
  73,
  74,
  78,
  79,
  86,
  88,
  89,
  100,
  103,
  111,
  137,
  166,
  176,
  213,
  247,
  317,
  349,
  350,
  351,
  352,
  353,
  354,
  355,
  356

In [ ]:
# change cohort nulls to 0
df.cohort = df.cohort.fillna(0)

In [ ]:
# drop 1 null in endpoint
df = df.dropna()

In [ ]:
# Remove '\t\t' character from col1
df.dates = df.dates.str.replace("\t\t", "")
df

In [ ]:
# check nulls
df.isnull().sum()

In [ ]:
pd.set_option('display.max_colwidth',1000)
# finding unique pages -- 2313 pages
df.endpoint.value_counts()

In [ ]:
js_result = df[df['endpoint'].str.startswith('javascript')]

In [ ]:
js_result.columns


In [ ]:
# rename cohort to id to match 
temp_df = df.copy()

In [ ]:
# cohort names ids
cohort_df = pd.read_csv("curriculum_logs_cohorts.csv")

In [ ]:
cohort_df.columns

In [ ]:
cohort_df = cohort_df.rename(columns = {"id": "cohort"})

In [ ]:
cohort_df

In [ ]:
temp_df

In [ ]:
df = pd.merge(cohort_df, temp_df,  on=['cohort'], how='inner')

In [ ]:
df

In [ ]:
# rename columns 'cohort', 'cohort_name', 'time', 'endpoint', 'user_id', 'cohort', 'ip' 'start_date', 'end_date', 'program_id'
df = df.drop(columns=[ 'slack',  'created_at', 'updated_at', 'deleted_at'])

In [ ]:
df.info()

In [ ]:
df.start_date = df.start_date.astype("datetime64")
df.end_date = df.end_date.astype("datetime64")

In [ ]:
# update dtype for dates to datetime64
df.dates = df.dates.astype("datetime64")

In [ ]:
df.info()

In [ ]:
def prepare_df():
    # read df
    df = w.acquire_df()
    # ----handle nulls
    # change cohort nulls to 0
    df.cohort = df.cohort.fillna(0)
    
    

    # drop 1 null in endpoint
    df = df.dropna()

    # Remove '\t\t' character from col1
    df.dates = df.dates.str.replace("\t\t", "")
    

    # read cohort df
    cohort_df = pd.read_csv("curriculum_logs_cohorts.csv")
    
    # ----- merge tables
    # rename id to cohort to match
    cohort_df = cohort_df.rename(columns = {"id": "cohort"})

    # merge on cohort
    df = pd.merge(cohort_df, df,  on=['cohort'], how='right')

    # drop columns
    df = df.drop(columns=[ 'slack',  'created_at', 'updated_at', 'deleted_at'])

    # update dtype for dates to datetime64
    df.start_date = df.start_date.astype("datetime64")
    df.end_date = df.end_date.astype("datetime64")
    df.dates = df.dates.astype("datetime64")

    return df  


In [ ]:
prepare_df()

In [ ]:
w.prepare_df()